In [1]:
# This code import external economic data into the dataset, including mltv, hpi factor and yield curve slope

In [2]:
# Should use the yearly data merged by the R script instead of the raw data from the website

In [3]:
import numpy as np
import pandas as pd

In [4]:
def run(year):
    read = str(year) + '_data.csv'
    df = pd.read_csv(read ,index_col= 0)
    df['60_days_delinq'] = ((pd.to_datetime(df['F60_DTE']) - pd.to_datetime(df['ORIG_DTE'])).map(lambda x: x.days) - 365*2<=0).map(int)
    df['90_days_delinq'] = ((pd.to_datetime(df['F90_DTE']) - pd.to_datetime(df['ORIG_DTE'])).map(lambda x: x.days) - 365*3<=0).map(int)
    df['prepay'] = (df['LAST_STAT'] == 'P').map(int)
    xy = ((df['LAST_STAT'] == 'S') | (df['LAST_STAT'] == 'R') | (df['LAST_STAT'] == 'F') ).map(int)
    abc = ((pd.to_datetime(df['LAST_DTE']) - pd.to_datetime(df['ORIG_DTE'])).map(lambda x: x.days) - 365*3<=0).map(int)
    df['default'] = xy*abc
    df['first_home_buyer'] = (df['FTHB_FLG'] == 'Y').map(int)
    df['loan_purpose_purchase']  = (df['PURPOSE'] == 'P').map(int)
    df['loan_purpose_no_cash_out_refinance']  = (df['PURPOSE'] == 'R').map(int)
    df['loan_purpose_cash_out_refinance']  = (df['PURPOSE'] == 'C').map(int)

    df['property_type_single_family'] = (df['PROP_TYP'] == 'SF').map(int)
    df['property_type_condo'] = (df['PROP_TYP'] == 'CO').map(int)
    df['property_type_coop'] = (df['PROP_TYP'] == 'CP').map(int)
    df['property_type_manufactured_housing'] = (df['PROP_TYP'] == 'MH').map(int)
    df['property_type_planed_urban_development'] = (df['PROP_TYP'] == 'PU').map(int)

    df['occupancy_status_principal'] = (df['OCC_STAT'] == 'P').map(int)
    df['occupancy_status_second'] = (df['OCC_STAT'] == 'S').map(int)
    df['occupancy_status_investor'] = (df['OCC_STAT'] == 'I').map(int)

    df['original_housing_value'] = 100*df['ORIG_AMT']/df['OLTV']

    avg_upb_dict = df.groupby('STATE').mean()['ORIG_AMT'].to_dict()
    df['Average_same_state_upb'] = df['STATE'].map(avg_upb_dict)
    df['loan_size'] = df['ORIG_AMT'] / df['Average_same_state_upb']
    df['modified'] = (df['MOD_FLAG'] == 'Y').map(int)

    df_hpi = pd.read_csv('hpi.csv',header = None)
    df_hpi.columns = ['ZIP','Year','Quarter','Index']   
    df_hpi['ZIP_Year_Quarter'] = list(zip(df_hpi['ZIP'],df_hpi['Year'],df_hpi['Quarter']))
    df_hpi.set_index('ZIP_Year_Quarter', inplace = True)
    index_dict = df_hpi['Index'].to_dict()

    # df_un = pd.read_csv('unemployment.csv',header = None)
    # df_un.dropna(inplace = True)
    # df_un.columns = ['MSA','Year','Month','Unemployment']
    # df_un['MSA_Year_Month'] = list(zip(df_un['MSA'],df_un['Year'],df_un['Month']))
    # df_un.set_index('MSA_Year_Month', inplace = True)
    # un_dict = df_un['Unemployment'].to_dict()

    df_1yr = pd.read_csv('1yrate.csv', header = None)
    df_10yr = pd.read_csv('10yrate.csv', header = None)
    df_1yr.columns = ['Date','Rate']
    df_10yr.columns = ['Date','Rate']
    df_1yr['Date'] = pd.to_datetime(df_1yr['Date'])
    df_10yr['Date'] = pd.to_datetime(df_10yr['Date'])
    df_1yr['Year'] = df_1yr['Date'].map(lambda x: x.year)
    df_1yr['Month'] = df_1yr['Date'].map(lambda x: x.month)
    df_10yr['Year'] = df_10yr['Date'].map(lambda x: x.year)
    df_10yr['Month'] = df_10yr['Date'].map(lambda x: x.month)
    df_1yr['Year_Month'] = list(zip(df_1yr['Year'],df_1yr['Month']))
    df_10yr['Year_Month'] = list(zip(df_10yr['Year'],df_10yr['Month']))
    df_1yr = df_1yr.groupby(by = 'Year_Month').mean()
    df_10yr = df_10yr.groupby(by = 'Year_Month').mean()
    dict_1yr = df_1yr['Rate'].to_dict()
    dict_10yr = df_10yr['Rate'].to_dict()

    df_ym = df['LAST_DTE'].str.extract(r'(?P<Year>\d\d\d\d)-(?P<Month>\d\d)')
    df['Year'] = df_ym['Year']
    df['Month'] = df_ym['Month']
    df['Year'] = pd.to_numeric(df['Year'])
    df['Month'] = pd.to_numeric(df['Month'])
    quarter_dict = {1:1,2:1,3:1,4:2,5:2,6:2,7:3,8:3,9:3,10:4,11:4,12:4}
    df['Quarter'] = df['Month'].map(quarter_dict)
    df['ZIP_Year_Quarter'] = list(zip(df['ZIP_3'],df['Year'],df['Quarter']))

    df_orig_ym = df['ORIG_DTE'].str.extract(r'(?P<Month>\d\d)/(?P<Year>\d\d\d\d)')
    df['Orig_Year'] = pd.to_numeric(df_orig_ym['Year'])
    df['Orig_Month'] = pd.to_numeric(df_orig_ym['Month'])
    df['Orig_Quarter'] = df['Orig_Month'].map(quarter_dict)
    df['ZIP_Orig_Year_Quarter'] = list(zip(df['ZIP_3'],df['Orig_Year'],df['Orig_Quarter']))

    df['Year'] = df['Orig_Year'] + 1
    df['ZIP_Year_Quarter'] = list(zip(df['ZIP_3'],df['Year'],df['Orig_Quarter']))

    df['current_hpi'] = df['ZIP_Year_Quarter'].map(index_dict)
    df['original_hpi'] = df['ZIP_Orig_Year_Quarter'].map(index_dict)
    df['hpi_factor'] = df['current_hpi']/df['original_hpi'] 
    df['mltv'] = (df['LAST_UPB']/df['hpi_factor'])/df['original_housing_value']

    df['LOAN_AGE'] = pd.to_datetime(df['LAST_DTE']) - pd.to_datetime(df['ORIG_DTE'])
    df['LOAN_AGE'] = df['LOAN_AGE'].map(lambda x: x.days)

    df['Year_Month'] = list(zip(df['Orig_Year'],df['Orig_Month']))
    df['1yr_rate'] = df['Year_Month'].map(dict_1yr)
    df['10yr_rate'] = df['Year_Month'].map(dict_10yr)
    df['slope'] = df['10yr_rate'] - df['1yr_rate']

    df['Yearr_Month'] = list(zip(df['Year'],df['Orig_Month']))
    df['1yrr_rate'] = df['Yearr_Month'].map(dict_1yr)
    df['10yrr_rate'] = df['Yearr_Month'].map(dict_10yr)
    df['slope1'] = df['10yrr_rate'] - df['1yrr_rate']

    df_total = df[['LOAN_ID', 
                     '60_days_delinq', '90_days_delinq',
                     'default', 'prepay',
                     'LAST_RT','LAST_UPB','LOAN_AGE',
                     'ORIG_RT','ORIG_AMT','OLTV','NUM_BO','DTI','CSCORE_B','NUM_UNIT',
                     'first_home_buyer',
                     'loan_purpose_purchase', 'loan_purpose_no_cash_out_refinance',
                     'loan_purpose_cash_out_refinance', 'property_type_single_family',
                     'property_type_condo', 'property_type_coop',
                     'property_type_manufactured_housing',
                     'property_type_planed_urban_development', 'occupancy_status_principal',
                     'occupancy_status_second', 'occupancy_status_investor',
                     'original_housing_value', 'Average_same_state_upb','loan_size',
                     'hpi_factor','mltv','slope','slope1','modified']]

    df_total.dropna(inplace = True)
    write = str(year) + '_model_data.csv'
    df_total.to_csv(write)
    
    return 0

In [5]:
run(2005)

/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [6]:
run(2006)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [7]:
run(2007)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [8]:
run(2008)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [14]:
run(2011)

ParserError: Error tokenizing data. C error: EOF inside string starting at row 3027

In [15]:
run(2012)

KeyboardInterrupt: 

In [11]:
run(2013)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [12]:
run(2014)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [13]:
run(2015)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3220: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0

In [21]:
run(2011)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0